In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install --upgrade pip 
!pip install "unsloth[cu126-torch260] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 5632 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2-0.5B-Instruct", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Qwen2 patching release 2024.6
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 12.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.6 patched 24 layers with 0 QKV layers, 24 O layers and 24 MLP layers.


In [4]:
from datasets import load_dataset
dataset = load_dataset("kyujinpy/KOpen-platypus")

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'data_source'],
        num_rows: 24926
    })
})

In [6]:
dataset['train'][0]

{'input': '',
 'output': '모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \\frac{1}{3}$, $P(B) = \\frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \\frac{1}{3} - frac{5}{12} = \\frac{12}{12} - frac{4}{12} - frac{5}{12} = \\frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \\frac{1}{4}$입니다.',
 'instruction': '보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.',
 'data_source': 'MATH/PRM-800K'}

In [7]:
train_data = dataset['train']

In [8]:
messages = [
    {"role": "user",
        "content": train_data[0]['instruction']
    }
]

chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
chat_message


'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.<|im_end|>\n<|im_start|>assistant\n'

In [9]:
def generate_prompt(data):
    output_texts = []
    messages = [
        {
            "role": "user",
            "content": data['instruction']
        },
        {
            "role": "assistant",
            "content": data['output']
        }
    ]

    chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    chat_message = chat_message.replace("system\nYou are a helpful assistant.", "system\n당신은 친절한 어시스턴트입니다.")
    output_texts.append(chat_message)
    return output_texts

In [10]:
chat_message = generate_prompt(train_data[0])
chat_message

['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.<|im_end|>\n<|im_start|>assistant\n모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \\frac{1}{3}$, $P(B) = \\frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \\frac{1}{3} - frac{5}{12} = \\frac{12}{12} - frac{4}{12} - frac{5}{12} = \\frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \\frac{1}{4}$입니다.<|im_end|>\n<eos>']

In [11]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        if len(input.strip()) > 0 :
            instruction = f"{instruction}\n\n{input}"
        messages = [
            {
                "role": "user",
                "content": instruction
            },
        ]
        messages.append(
            {
                "role": "assistant",
                "content": output
            }
        )            
        chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        chat_message = chat_message.replace("system\nYou are a helpful assistant.", "system\n당신은 친절한 어시스턴트입니다.")
        texts.append(chat_message)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("kyujinpy/KOpen-platypus", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import math

batch_size = 2
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = batch_size,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = math.ceil(len(dataset) / batch_size),
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [13]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3060. Max memory = 12.0 GB.
0.666 GB of memory reserved.


In [ ]:
import wandb
wandb.init(mode="disabled")

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 24,926 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 8,798,208


Step,Training Loss
100,1.616100
200,1.485600
300,1.433400


KeyboardInterrupt: 

In [15]:
from transformers import TextStreamer, GenerationConfig

streamer = TextStreamer(tokenizer)

def gen(prompt):
    generation_config = GenerationConfig(
        temperature = 1.31,
        max_new_tokens = 512,
        exponential_decay_length_penalty = (512, 1.03),
        eos_token_id = tokenizer.eos_token_id,
        repetition_penalty = 1.17,
        do_sample = True,
        top_k = 49,
        top_p = 0.14,
        min_length = 5,
        use_cache = True,
        return_dict_in_generate = True
    )
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to("cuda"),
        generation_config=generation_config,
        streamer=streamer
    )

In [16]:
messages = [
    {"role": "user", 
     "content": "이순신 장군에 대해 설명해주세요."},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True) 

gen(prompt)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
이순신 장군에 대해 설명해주세요.<|im_end|>
<|im_start|>assistant
이순신은 조선의 전통적인 군인으로, 18세기 중반부터 서양에서 태어났습니다. 그는 농부를 운영하고 싶었고, 이래서 경제적으로 많은 성공을 거두었습니다.

이순신은 빠른 계획과 지속적인 노력으로 인생을 향한 것이었다면 이들에 대한 고려가 없거나, 다른 사람들의 도움이 될 수 있는 방법이 있으면 좋았겠지만, 그녀는 자신만의 방식으로 일할 것을 원했습니다.

그러나 그녀는 자신의 생활을 더욱 즐거워졌다는 결론을 

KeyboardInterrupt: 